In [21]:
import cv2
import matplotlib.pyplot as plt
import os
import paddle as paddle
import paddle.fluid as fluid
import PIL.Image as Image
import random
import numpy as np
import time
from htgCrap import htgCrap
from werkzeug.utils import secure_filename
from flask import Flask, request
from flask_cors import CORS
import uuid
import json

In [22]:
def load_model(model_id):
    model_net_path = 'models/infer_model/'+str(model_id)+'/'
    place = fluid.CPUPlace()
    exe = fluid.Executor(place)
    [infer_program, feeded_var_names, target_var] = fluid.io.load_inference_model(dirname=model_net_path, executor=exe)
    return exe, infer_program , feeded_var_names , target_var

def model_infer(image,*args):
    exe, infer_program , feeded_var_names , target_var = args[0]
    image = np.array(image).reshape(-1,1,64,64).astype('float32')
    image = (image) /255.0
    result = exe.run(program = infer_program,
                    feed = {feeded_var_names[0]:image},
                     fetch_list = target_var
                    )
    return result

In [23]:
app = Flask(__name__)
CORS(app)
args = load_model(36)

In [24]:
def inferImage(image_path):
    itg = htgCrap(image_path)
    FL = itg.crapNumber(64)
    formula_list = []
    for fl in FL:
        result = model_infer(fl,args)
        re_str = ""
        math_map = ['0','1','2','3','4','5','6','7','8','9','^','+','-','*','/',')','(','=','Z','a','y']
        for i in result[0]:
            re = np.argmax(i)
            if re == 18:
                continue
            re_str+=math_map[re]
        formula_list.append(re_str)
    return formula_list

In [25]:
@app.route('/infer', methods=['POST'])
def infer():
    f = request.files['img1']
    # 保存图片
    save_father_path = 'images'
    img_path = os.path.join(save_father_path, str(uuid.uuid1()) + '.' + secure_filename(f.filename).split('.')[-1])
    if not os.path.exists(save_father_path):
        os.makedirs(save_father_path)
    f.save(img_path)
    r = inferImage(img_path)
    os.remove(img_path)
    r = json.dumps(r)
    r = '{"result": '+str(r)+'}'
    return r

In [26]:
ip1 = '127.0.0.1'
ip2 = '172.16.7.238'
if __name__ == '__main__':
    # 启动服务，并指定端口号
    app.run(host = str(ip2),port=870)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
